## Importing Packages

In [1]:
%pip install tensorflow
%pip install statsmodels
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from itertools import product
from datetime import timedelta

In [100]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## Loading Dataset

In [101]:
df = pd.read_csv('final_merged_copper_dataset.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df["Date"] = pd.to_datetime(df["Date"])
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-04,0.8480,1,2000,14096.033,165.600,3.376857,59.47000
1,2000-01-05,0.8565,1,2000,14096.033,165.600,3.376857,59.47000
2,2000-01-06,0.8530,1,2000,14096.033,165.600,3.376857,59.47000
3,2000-01-07,0.8540,1,2000,14096.033,165.600,3.376857,59.47000
4,2000-01-10,0.8465,1,2000,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...,...
6051,2023-10-16,3.5820,10,2023,19374.000,307.789,6.600000,168.40467
6052,2023-10-17,3.5785,10,2023,19374.000,307.789,6.600000,168.40467
6053,2023-10-18,3.5870,10,2023,19374.000,307.789,6.600000,168.40467
6054,2023-10-19,3.6010,10,2023,19374.000,307.789,6.600000,168.40467


In [102]:
df.to_csv("df.csv",index=False)

In [5]:
last_date = df['Date'].max().date()
last_date

datetime.date(2023, 10, 20)

In [6]:
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-04,0.8480,1,2000,14096.033,165.600,3.376857,59.47000
1,2000-01-05,0.8565,1,2000,14096.033,165.600,3.376857,59.47000
2,2000-01-06,0.8530,1,2000,14096.033,165.600,3.376857,59.47000
3,2000-01-07,0.8540,1,2000,14096.033,165.600,3.376857,59.47000
4,2000-01-10,0.8465,1,2000,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...,...
6051,2023-10-16,3.5820,10,2023,19374.000,307.789,6.600000,168.40467
6052,2023-10-17,3.5785,10,2023,19374.000,307.789,6.600000,168.40467
6053,2023-10-18,3.5870,10,2023,19374.000,307.789,6.600000,168.40467
6054,2023-10-19,3.6010,10,2023,19374.000,307.789,6.600000,168.40467


In [7]:
LOOKBACK = 7
data = pd.DataFrame({'Date': df["Date"].copy(),
                     'Price': df["Price"].copy()})
# Select the "Date" column and store it in the variable 'train_dates'.
dates = data["Date"]

# Exclude the first 'LOOKBACK' dates from 'train_dates' to create a time window.
dates = dates[LOOKBACK:]
data

,Date,Price
0,2000-01-04,0.8480
1,2000-01-05,0.8565
2,2000-01-06,0.8530
3,2000-01-07,0.8540
4,2000-01-10,0.8465
...,...,...
6051,2023-10-16,3.5820
6052,2023-10-17,3.5785
6053,2023-10-18,3.5870
6054,2023-10-19,3.6010


In [8]:
data

,Date,Price
0,2000-01-04,0.8480
1,2000-01-05,0.8565
2,2000-01-06,0.8530
3,2000-01-07,0.8540
4,2000-01-10,0.8465
...,...,...
6051,2023-10-16,3.5820
6052,2023-10-17,3.5785
6053,2023-10-18,3.5870
6054,2023-10-19,3.6010


In [9]:
dates.reset_index(drop=True, inplace=True)

In [10]:
dates

0      2000-01-13
1      2000-01-14
2      2000-01-18
3      2000-01-19
4      2000-01-20
          ...    
6044   2023-10-16
6045   2023-10-17
6046   2023-10-18
6047   2023-10-19
6048   2023-10-20
Name: Date, Length: 6049, dtype: datetime64[ns]

In [11]:
from copy import deepcopy as dc


def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)

    #df.set_index('Date', inplace=True)

    for i in range(1, n_steps + 1):
        df[f'Price(t-{i})'] = df['Price'].shift(i)

    df.dropna(inplace=True)

    return df


shifted_df = prepare_dataframe_for_lstm(data, LOOKBACK)
shifted_df = shifted_df.reset_index(drop=True)
df = df.drop(df.index[-7:])
df

,Date,Price,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-04,0.8480,1,2000,14096.033,165.600,3.376857,59.47000
1,2000-01-05,0.8565,1,2000,14096.033,165.600,3.376857,59.47000
2,2000-01-06,0.8530,1,2000,14096.033,165.600,3.376857,59.47000
3,2000-01-07,0.8540,1,2000,14096.033,165.600,3.376857,59.47000
4,2000-01-10,0.8465,1,2000,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...,...
6044,2023-10-05,3.5520,10,2023,19374.000,307.789,6.600000,168.40467
6045,2023-10-06,3.6275,10,2023,19374.000,307.789,6.600000,168.40467
6046,2023-10-09,3.6460,10,2023,19374.000,307.789,6.600000,168.40467
6047,2023-10-10,3.6340,10,2023,19374.000,307.789,6.600000,168.40467


In [12]:
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7)
0,2000-01-13,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,0.8480
1,2000-01-14,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565
2,2000-01-18,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530
3,2000-01-19,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540
4,2000-01-20,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465
...,...,...,...,...,...,...,...,...,...
6044,2023-10-16,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275,3.5520
6045,2023-10-17,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275
6046,2023-10-18,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460
6047,2023-10-19,3.6010,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340


In [13]:
shifted_df["Real_GDP"] = df["Real_GDP"]
shifted_df["CPI"] = df["CPI"]
shifted_df["inflation_rate"] = df["inflation_rate"]
shifted_df["PALLFNFINDEXM"] = df["PALLFNFINDEXM"]
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-13,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,165.600,3.376857,59.47000
1,2000-01-14,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,165.600,3.376857,59.47000
2,2000-01-18,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,165.600,3.376857,59.47000
3,2000-01-19,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,165.600,3.376857,59.47000
4,2000-01-20,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044,2023-10-16,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275,3.5520,19374.000,307.789,6.600000,168.40467
6045,2023-10-17,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275,19374.000,307.789,6.600000,168.40467
6046,2023-10-18,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,19374.000,307.789,6.600000,168.40467
6047,2023-10-19,3.6010,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,19374.000,307.789,6.600000,168.40467


In [14]:
prices_before_scaling_as_np = shifted_df[["Price","Price(t-1)","Price(t-2)","Price(t-3)","Price(t-4)","Price(t-5)","Price(t-6)","Price(t-7)"]].values
prices_before_scaling_as_np.shape

(6049, 8)

In [15]:
prices_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_prices = prices_scaler.fit_transform(prices_before_scaling_as_np)
scaled_prices

array([[-0.88542389, -0.88103488, -0.89142989, ..., -0.88704089,
        -0.88542389, -0.88935089],
       [-0.88265188, -0.88542389, -0.88103488, ..., -0.88657889,
        -0.88704089, -0.88542389],
       [-0.88103488, -0.88265188, -0.88542389, ..., -0.89004389,
        -0.88657889, -0.88704089],
       ...,
       [ 0.37606838,  0.37214137,  0.37375837, ...,  0.38761839,
         0.3977824 ,  0.4033264 ],
       [ 0.38253638,  0.37606838,  0.37214137, ...,  0.37791638,
         0.38761839,  0.3977824 ],
       [ 0.36498036,  0.38253638,  0.37606838, ...,  0.36867637,
         0.37791638,  0.38761839]])

In [16]:
scaled_prices.shape

(6049, 8)

In [17]:
GDP_before_scaling_as_np = shifted_df["Real_GDP"].to_numpy()
CPI_before_scaling_as_np = shifted_df["CPI"].to_numpy()
inflation_rate_before_scaling_as_np = shifted_df["inflation_rate"].to_numpy()
PALLFNFINDEXM_before_scaling_as_np = shifted_df["PALLFNFINDEXM"].to_numpy()

In [18]:
GDP_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_GDP = GDP_scaler.fit_transform(GDP_before_scaling_as_np.reshape(-1, 1))

In [19]:
scaled_GDP

array([[-1.        ],
       [-1.        ],
       [-1.        ],
       ...,
       [ 0.36628638],
       [ 0.36628638],
       [ 0.36628638]])

In [20]:
CPI_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_CPI = CPI_scaler.fit_transform(CPI_before_scaling_as_np.reshape(-1, 1))

In [21]:
scaled_CPI

array([[-1.],
       [-1.],
       [-1.],
       ...,
       [ 1.],
       [ 1.],
       [ 1.]])

In [22]:
IR_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_IR = IR_scaler.fit_transform(inflation_rate_before_scaling_as_np.reshape(-1, 1))

In [23]:
scaled_IR

array([[-0.10690381],
       [-0.10690381],
       [-0.10690381],
       ...,
       [ 0.66433555],
       [ 0.66433555],
       [ 0.66433555]])

In [24]:
PALLFNFINDEXM_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_PALLFNFINDEXM = PALLFNFINDEXM_scaler.fit_transform(PALLFNFINDEXM_before_scaling_as_np.reshape(-1, 1))

In [25]:
scaled_PALLFNFINDEXM

array([[-0.88354852],
       [-0.88354852],
       [-0.88354852],
       ...,
       [ 0.24106177],
       [ 0.24106177],
       [ 0.24106177]])

In [26]:

stacked_np = np.hstack((scaled_prices, scaled_GDP,scaled_CPI,scaled_IR,scaled_PALLFNFINDEXM))

# Display the combined NumPy array
stacked_np.shape


(6049, 12)

In [27]:
scaled_prices.shape

(6049, 8)

In [28]:
scaled_df = shifted_df.drop(['Date'], axis=1)
scaled_df

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,165.600,3.376857,59.47000
1,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,165.600,3.376857,59.47000
2,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,165.600,3.376857,59.47000
3,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,165.600,3.376857,59.47000
4,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...,...,...,...,...,...
6044,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275,3.5520,19374.000,307.789,6.600000,168.40467
6045,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275,19374.000,307.789,6.600000,168.40467
6046,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,19374.000,307.789,6.600000,168.40467
6047,3.6010,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,19374.000,307.789,6.600000,168.40467


In [29]:
flipped_df = shifted_df[shifted_df.columns[::-1]]
flipped_df = flipped_df.drop(['Date'], axis=1)
flipped_df

,PALLFNFINDEXM,inflation_rate,CPI,Real_GDP,Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1),Price
0,59.47000,3.376857,165.600,14096.033,0.8480,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565
1,59.47000,3.376857,165.600,14096.033,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625
2,59.47000,3.376857,165.600,14096.033,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660
3,59.47000,3.376857,165.600,14096.033,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825
4,59.47000,3.376857,165.600,14096.033,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730
...,...,...,...,...,...,...,...,...,...,...,...,...
6044,168.40467,6.600000,307.789,19374.000,3.5520,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820
6045,168.40467,6.600000,307.789,19374.000,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785
6046,168.40467,6.600000,307.789,19374.000,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870
6047,168.40467,6.600000,307.789,19374.000,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870,3.6010


In [30]:
flipped_df.reset_index(drop=True, inplace=True)

In [31]:
df.drop(columns=['Price'], inplace=True)

In [32]:
new_df = pd.concat([df, flipped_df], axis=1)

In [33]:
new_df

,Date,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,PALLFNFINDEXM,inflation_rate,CPI,Real_GDP,Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1),Price
0,2000-01-04,1,2000,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8480,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565
1,2000-01-05,1,2000,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625
2,2000-01-06,1,2000,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660
3,2000-01-07,1,2000,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825
4,2000-01-10,1,2000,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044,2023-10-05,10,2023,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.5520,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820
6045,2023-10-06,10,2023,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785
6046,2023-10-09,10,2023,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870
6047,2023-10-10,10,2023,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870,3.6010


In [34]:
new_df.drop(columns=['Month', 'Year'], inplace=True)
new_df

,Date,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM,PALLFNFINDEXM,inflation_rate,CPI,Real_GDP,Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1),Price
0,2000-01-04,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8480,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565
1,2000-01-05,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625
2,2000-01-06,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660
3,2000-01-07,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825
4,2000-01-10,14096.033,165.600,3.376857,59.47000,59.47000,3.376857,165.600,14096.033,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044,2023-10-05,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.5520,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820
6045,2023-10-06,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785
6046,2023-10-09,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870
6047,2023-10-10,19374.000,307.789,6.600000,168.40467,168.40467,6.600000,307.789,19374.000,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870,3.6010


In [35]:
flipped_df

,PALLFNFINDEXM,inflation_rate,CPI,Real_GDP,Price(t-7),Price(t-6),Price(t-5),Price(t-4),Price(t-3),Price(t-2),Price(t-1),Price
0,59.47000,3.376857,165.600,14096.033,0.8480,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565
1,59.47000,3.376857,165.600,14096.033,0.8565,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625
2,59.47000,3.376857,165.600,14096.033,0.8530,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660
3,59.47000,3.376857,165.600,14096.033,0.8540,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825
4,59.47000,3.376857,165.600,14096.033,0.8465,0.8435,0.8660,0.8565,0.8625,0.8660,0.8825,0.8730
...,...,...,...,...,...,...,...,...,...,...,...,...
6044,168.40467,6.600000,307.789,19374.000,3.5520,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820
6045,168.40467,6.600000,307.789,19374.000,3.6275,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785
6046,168.40467,6.600000,307.789,19374.000,3.6460,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870
6047,168.40467,6.600000,307.789,19374.000,3.6340,3.6120,3.5910,3.5710,3.5820,3.5785,3.5870,3.6010


In [36]:
flipped_df_before_scaling_as_np = flipped_df.to_numpy()
flipped_df_before_scaling_as_np

array([[ 59.47      ,   3.37685727, 165.6       , ...,   0.8435    ,
          0.866     ,   0.8565    ],
       [ 59.47      ,   3.37685727, 165.6       , ...,   0.866     ,
          0.8565    ,   0.8625    ],
       [ 59.47      ,   3.37685727, 165.6       , ...,   0.8565    ,
          0.8625    ,   0.866     ],
       ...,
       [168.40467032,   6.6       , 307.789     , ...,   3.582     ,
          3.5785    ,   3.587     ],
       [168.40467032,   6.6       , 307.789     , ...,   3.5785    ,
          3.587     ,   3.601     ],
       [168.40467032,   6.6       , 307.789     , ...,   3.587     ,
          3.601     ,   3.563     ]])

In [37]:
price_scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_prices = price_scaler.fit_transform(flipped_df_before_scaling_as_np)

In [38]:
scaled_prices

array([[-0.88354852, -0.10690381, -1.        , ..., -0.89142989,
        -0.88103488, -0.88542389],
       [-0.88354852, -0.10690381, -1.        , ..., -0.88103488,
        -0.88542389, -0.88265188],
       [-0.88354852, -0.10690381, -1.        , ..., -0.88542389,
        -0.88265188, -0.88103488],
       ...,
       [ 0.24106177,  0.66433555,  1.        , ...,  0.37375837,
         0.37214137,  0.37606838],
       [ 0.24106177,  0.66433555,  1.        , ...,  0.37214137,
         0.37606838,  0.38253638],
       [ 0.24106177,  0.66433555,  1.        , ...,  0.37606838,
         0.38253638,  0.36498036]])

In [39]:
result = np.concatenate((scaled_GDP, scaled_CPI, scaled_IR, scaled_PALLFNFINDEXM, scaled_prices), axis=1)
result

array([[-1.        , -1.        , -0.10690381, ..., -0.89142989,
        -0.88103488, -0.88542389],
       [-1.        , -1.        , -0.10690381, ..., -0.88103488,
        -0.88542389, -0.88265188],
       [-1.        , -1.        , -0.10690381, ..., -0.88542389,
        -0.88265188, -0.88103488],
       ...,
       [ 0.36628638,  1.        ,  0.66433555, ...,  0.37375837,
         0.37214137,  0.37606838],
       [ 0.36628638,  1.        ,  0.66433555, ...,  0.37214137,
         0.37606838,  0.38253638],
       [ 0.36628638,  1.        ,  0.66433555, ...,  0.37606838,
         0.38253638,  0.36498036]])

In [40]:
X = result[:, 0:11]
y = result[:, 11]
X = X.astype(np.float32)
y = y.astype(np.float32)
X.dtype

dtype('float32')

In [41]:
X

array([[-1.        , -1.        , -0.10690381, ..., -0.8893509 ,
        -0.8854239 , -0.8870409 ],
       [-1.        , -1.        , -0.10690381, ..., -0.8854239 ,
        -0.8870409 , -0.88657886],
       [-1.        , -1.        , -0.10690381, ..., -0.8870409 ,
        -0.88657886, -0.8900439 ],
       ...,
       [ 0.36628637,  1.        ,  0.66433555, ...,  0.4033264 ,
         0.3977824 ,  0.3876184 ],
       [ 0.36628637,  1.        ,  0.66433555, ...,  0.3977824 ,
         0.3876184 ,  0.37791637],
       [ 0.36628637,  1.        ,  0.66433555, ...,  0.3876184 ,
         0.37791637,  0.36867636]], dtype=float32)

In [42]:
split_index = int(len(X) * 0.80)
split_date = dates[split_index]

X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = dates[:split_index]
test_dates = dates[split_index:]
test_dates

4839   2019-03-29
4840   2019-04-01
4841   2019-04-02
4842   2019-04-03
4843   2019-04-04
          ...    
6044   2023-10-16
6045   2023-10-17
6046   2023-10-18
6047   2023-10-19
6048   2023-10-20
Name: Date, Length: 1210, dtype: datetime64[ns]

In [43]:
X_train

array([[-1.        , -1.        , -0.10690381, ..., -0.8893509 ,
        -0.8854239 , -0.8870409 ],
       [-1.        , -1.        , -0.10690381, ..., -0.8854239 ,
        -0.8870409 , -0.88657886],
       [-1.        , -1.        , -0.10690381, ..., -0.8870409 ,
        -0.88657886, -0.8900439 ],
       ...,
       [ 0.7074943 ,  0.24625674, -0.48129538, ...,  0.06144606,
         0.06283206,  0.06930007],
       [ 0.7074943 ,  0.24625674, -0.48129538, ...,  0.06283206,
         0.06930007,  0.06837607],
       [ 0.7074943 ,  0.24625674, -0.48129538, ...,  0.06930007,
         0.06837607,  0.06144606]], dtype=float32)

In [44]:
X_train.shape

(4839, 11)

In [45]:
X = X.reshape((-1, LOOKBACK+4, 1))
X_test = X_test.reshape((-1, LOOKBACK+4, 1))

y = y.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

In [46]:
X = torch.tensor(X).float()
y = torch.tensor(y).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

#X_test.dtype

In [47]:
X_test.shape

torch.Size([1210, 11, 1])

In [48]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_dataset = TimeSeriesDataset(X, y)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [49]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [50]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([8, 11, 1]) torch.Size([8, 1])


In [51]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 4, 1)
model.to(device)
model

LSTM(
  (lstm): LSTM(1, 4, batch_first=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [52]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.5f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()

In [53]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)

    print('Val Loss: {0:.5f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [54]:
learning_rate = 0.001
num_epochs = 30
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#Adam does the gradient descent

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1
Batch 100, Loss: 0.21949
Batch 200, Loss: 0.13277
Batch 300, Loss: 0.03340
Batch 400, Loss: 0.01435
Batch 500, Loss: 0.01074
Batch 600, Loss: 0.00842
Batch 700, Loss: 0.00653

Val Loss: 0.00895
***************************************************

Epoch: 2
Batch 100, Loss: 0.00433
Batch 200, Loss: 0.00346
Batch 300, Loss: 0.00222
Batch 400, Loss: 0.00176
Batch 500, Loss: 0.00136
Batch 600, Loss: 0.00120
Batch 700, Loss: 0.00104

Val Loss: 0.00124
***************************************************

Epoch: 3
Batch 100, Loss: 0.00095
Batch 200, Loss: 0.00102
Batch 300, Loss: 0.00087
Batch 400, Loss: 0.00069
Batch 500, Loss: 0.00083
Batch 600, Loss: 0.00066
Batch 700, Loss: 0.00071

Val Loss: 0.00095
***************************************************

Epoch: 4
Batch 100, Loss: 0.00076
Batch 200, Loss: 0.00068
Batch 300, Loss: 0.00069
Batch 400, Loss: 0.00069
Batch 500, Loss: 0.00073
Batch 600, Loss: 0.00069
Batch 700, Loss: 0.00069

Val Loss: 0.00106
*****************************

In [55]:
with torch.no_grad():
    predicted = model(X_train[:].to(device)).to('cpu').numpy()

plt.plot(train_dates, y_train, label='Actual Price')
plt.plot(train_dates, predicted, label='Predicted Price')

plt.xlabel('Dates')
plt.ylabel('Price')
plt.legend()
plt.show()


AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [ ]:
train_predictions = predicted.flatten()

dummies = np.zeros((X_train.shape[0], LOOKBACK+1))
dummies[:, 0] = train_predictions
dummies = price_scaler.inverse_transform(dummies)

train_predictions = dc(dummies[:, 0])
train_predictions

In [ ]:
dummies = np.zeros((X_train.shape[0], LOOKBACK+1))
dummies[:, 0] = y_train.flatten()
dummies = price_scaler.inverse_transform(dummies)
new_y_train = dc(dummies[:, 0])
new_y_train

In [ ]:
plt.plot(train_dates,new_y_train,  label='Actual Price')
plt.plot(train_dates,train_predictions, label='Predicted Price')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
test_predictions = model(X_test.to(device)).detach().cpu().numpy().flatten()

dummies = np.zeros((X_test.shape[0], LOOKBACK+1))
dummies[:, 0] = test_predictions
dummies = price_scaler.inverse_transform(dummies)

test_predictions = dc(dummies[:, 0])
test_predictions

In [ ]:
dummies = np.zeros((X_test.shape[0], LOOKBACK+1))
dummies[:, 0] = y_test.flatten()
dummies = price_scaler.inverse_transform(dummies)

new_y_test = dc(dummies[:, 0])
new_y_test
squared_differences = (new_y_test - test_predictions)**2

# Calculate the Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean(squared_differences))
#rmse
absolute_differences = np.abs(new_y_test - test_predictions)

# Calculate the Mean Absolute Error (MAE)
mae = np.mean(absolute_differences)
rmse

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(test_dates, new_y_test, label='Actual Price')
plt.plot(test_dates,test_predictions, label='Predicted Price')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
test_dates

In [ ]:
"""# Instantiate your model
model = LSTM(1, 4, 1)

# Save the model to a .pkl (pickle) file
model_file_path = 'LSTM.pkl'
with open(model_file_path, 'wb') as file:
    pickle.dump(model, file)"""

In [89]:
torch.save(model, 'model.pt')

In [ ]:
m1 = torch.load('model.pt')

In [ ]:
m1

In [ ]:
model

In [56]:
start_date = last_date+timedelta(days=1)
end_date = "2024-01-25"
forecast_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a DataFrame with the date column
forecast_set = pd.DataFrame({'Date': forecast_dates})
forecast_set['Date'] = pd.to_datetime(forecast_set['Date'])
forecast_set

,Date
0,2023-10-21
1,2023-10-22
2,2023-10-23
3,2023-10-24
4,2023-10-25
...,...
92,2024-01-21
93,2024-01-22
94,2024-01-23
95,2024-01-24


In [59]:
forecast_dates.shape

(97,)

In [91]:
stacked_np.dtype

dtype('float64')

In [90]:
np.save('stacked_np.npy', stacked_np)

In [57]:
shifted_df_as_np_flipped = np.fliplr(stacked_np)
shifted_df_as_np_flipped

array([[-0.88354852, -0.10690381, -1.        , ..., -0.89142989,
        -0.88103488, -0.88542389],
       [-0.88354852, -0.10690381, -1.        , ..., -0.88103488,
        -0.88542389, -0.88265188],
       [-0.88354852, -0.10690381, -1.        , ..., -0.88542389,
        -0.88265188, -0.88103488],
       ...,
       [ 0.24106177,  0.66433555,  1.        , ...,  0.37375837,
         0.37214137,  0.37606838],
       [ 0.24106177,  0.66433555,  1.        , ...,  0.37214137,
         0.37606838,  0.38253638],
       [ 0.24106177,  0.66433555,  1.        , ...,  0.37606838,
         0.38253638,  0.36498036]])

In [60]:
shifted_df_as_tensor = torch.tensor(shifted_df_as_np_flipped.copy())
torch.set_printoptions(precision=6)
last = shifted_df_as_tensor[-1][1:]
last = last.unsqueeze(0).unsqueeze(-1)
last.shape

torch.Size([1, 11, 1])

In [61]:
shifted_df_as_tensor.shape

torch.Size([6049, 12])

In [93]:
shifted_df

,Date,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-13,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,0.8480,14096.033,165.600,3.376857,59.47000
1,2000-01-14,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,0.8565,14096.033,165.600,3.376857,59.47000
2,2000-01-18,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,0.8530,14096.033,165.600,3.376857,59.47000
3,2000-01-19,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,0.8540,14096.033,165.600,3.376857,59.47000
4,2000-01-20,0.8730,0.8825,0.8660,0.8625,0.8565,0.8660,0.8435,0.8465,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044,2023-10-16,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275,3.5520,19374.000,307.789,6.600000,168.40467
6045,2023-10-17,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,3.6275,19374.000,307.789,6.600000,168.40467
6046,2023-10-18,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,3.6460,19374.000,307.789,6.600000,168.40467
6047,2023-10-19,3.6010,3.5870,3.5785,3.5820,3.5710,3.5910,3.6120,3.6340,19374.000,307.789,6.600000,168.40467


In [94]:
shifted_df.to_csv('shifted_df.csv', index=False, float_format='%.5f')

In [62]:
shifted_df_no_date = shifted_df.drop(['Date'], axis=1)

In [103]:
import joblib
joblib.dump(prices_scaler, 'prices_scaler.pkl')

['prices_scaler.pkl']

In [63]:
for i in range(forecast_dates.shape[0]):
    last = shifted_df_as_tensor[-1][1:]
    last = last.unsqueeze(0).unsqueeze(-1).float()
    with torch.no_grad():
        predicted = model(last.to(device)).to('cpu').numpy()
    row = torch.cat([shifted_df_as_tensor[-1][1:],torch.tensor(predicted).reshape(1)],dim=0).unsqueeze(0)
    quick_reverse = prices_scaler.inverse_transform(row[:,4:].to('cpu').numpy())
    new_row = {'Price':quick_reverse[0][7], 
               'Price(t-1)':quick_reverse[0][6],
               'Price(t-2)':quick_reverse[0][5],
               'Price(t-3)':quick_reverse[0][4],
               'Price(t-4)':quick_reverse[0][3],
               'Price(t-5)':quick_reverse[0][2],
               'Price(t-6)':quick_reverse[0][1],
               'Price(t-7)':quick_reverse[0][0],
               'Real_GDP':df.iloc[-1]["Real_GDP"],
               'CPI':df.iloc[-1]["CPI"],
               'inflation_rate':df.iloc[-1]["inflation_rate"],
               'PALLFNFINDEXM':df.iloc[-1]["PALLFNFINDEXM"]
               }
    shifted_df_as_tensor = torch.cat((shifted_df_as_tensor, row),dim=0)
    shifted_df_no_date = shifted_df_no_date._append(new_row,ignore_index=True)
    print(i)
print(shifted_df_no_date) 
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
         Price  Price(t-1)  Price(t-2)  Price(t-3)  Price(t-4)  Price(t-5)  \
0     0.856500    0.866000    0.843500    0.846500    0.854000    0.853000   
1     0.862500    0.856500    0.866000    0.843500    0.846500    0.854000   
2     0.866000    0.862500    0.856500    0.866000    0.843500    0.846500   
3     0.882500    0.866000    0.862500    0.856500    0.866000    0.843500   
4     0.873000    0.882500    0.866000    0.862500    0.856500    0.866000   
...        ...         ...         ...         ...         ...         ...   
6141  2.519781    2.527894    2.536108    2.544424    2.552841    2.561361   
6142  2.511768    2.519781    2.527894    2.536108    2.544424    2.552841   
6143  2.503856   

In [98]:
df

,Date,Month,Year,Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,2000-01-04,1,2000,14096.033,165.600,3.376857,59.47000
1,2000-01-05,1,2000,14096.033,165.600,3.376857,59.47000
2,2000-01-06,1,2000,14096.033,165.600,3.376857,59.47000
3,2000-01-07,1,2000,14096.033,165.600,3.376857,59.47000
4,2000-01-10,1,2000,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...
6044,2023-10-05,10,2023,19374.000,307.789,6.600000,168.40467
6045,2023-10-06,10,2023,19374.000,307.789,6.600000,168.40467
6046,2023-10-09,10,2023,19374.000,307.789,6.600000,168.40467
6047,2023-10-10,10,2023,19374.000,307.789,6.600000,168.40467


In [64]:
shifted_df_as_tensor.shape

torch.Size([6146, 12])

In [65]:
final_set = prices_scaler.inverse_transform(shifted_df_as_tensor[:,4:].to('cpu').numpy())
final_set

array([[0.848     , 0.8565    , 0.853     , ..., 0.8435    , 0.866     ,
        0.8565    ],
       [0.8565    , 0.853     , 0.854     , ..., 0.866     , 0.8565    ,
        0.8625    ],
       [0.853     , 0.854     , 0.8465    , ..., 0.8565    , 0.8625    ,
        0.866     ],
       ...,
       [2.56136099, 2.55284122, 2.54442367, ..., 2.51978103, 2.51176849,
        2.50385596],
       [2.55284122, 2.54442367, 2.53610812, ..., 2.51176849, 2.50385596,
        2.49604281],
       [2.54442367, 2.53610812, 2.52789404, ..., 2.50385596, 2.49604281,
        2.48832847]])

In [66]:
shifted_df_no_date

,Price,Price(t-1),Price(t-2),Price(t-3),Price(t-4),Price(t-5),Price(t-6),Price(t-7),Real_GDP,CPI,inflation_rate,PALLFNFINDEXM
0,0.856500,0.866000,0.843500,0.846500,0.854000,0.853000,0.856500,0.848000,14096.033,165.600,3.376857,59.47000
1,0.862500,0.856500,0.866000,0.843500,0.846500,0.854000,0.853000,0.856500,14096.033,165.600,3.376857,59.47000
2,0.866000,0.862500,0.856500,0.866000,0.843500,0.846500,0.854000,0.853000,14096.033,165.600,3.376857,59.47000
3,0.882500,0.866000,0.862500,0.856500,0.866000,0.843500,0.846500,0.854000,14096.033,165.600,3.376857,59.47000
4,0.873000,0.882500,0.866000,0.862500,0.856500,0.866000,0.843500,0.846500,14096.033,165.600,3.376857,59.47000
...,...,...,...,...,...,...,...,...,...,...,...,...
6141,2.519781,2.527894,2.536108,2.544424,2.552841,2.561361,2.569983,2.578708,19374.000,307.789,6.600000,168.40467
6142,2.511768,2.519781,2.527894,2.536108,2.544424,2.552841,2.561361,2.569983,19374.000,307.789,6.600000,168.40467
6143,2.503856,2.511768,2.519781,2.527894,2.536108,2.544424,2.552841,2.561361,19374.000,307.789,6.600000,168.40467
6144,2.496043,2.503856,2.511768,2.519781,2.527894,2.536108,2.544424,2.552841,19374.000,307.789,6.600000,168.40467


In [80]:
display_df = pd.DataFrame(shifted_df_no_date["Price"])
display_df

,Price
0,0.856500
1,0.862500
2,0.866000
3,0.882500
4,0.873000
...,...
6141,2.519781
6142,2.511768
6143,2.503856
6144,2.496043


In [81]:
display_df = display_df.iloc[6049:]
display_df

,Price
6049,3.552102
6050,3.540825
6051,3.529284
6052,3.517669
6053,3.506003
...,...
6141,2.519781
6142,2.511768
6143,2.503856
6144,2.496043


In [82]:
display_df = display_df.reset_index(drop=True)

In [83]:
display_df

,Price
0,3.552102
1,3.540825
2,3.529284
3,3.517669
4,3.506003
...,...
92,2.519781
93,2.511768
94,2.503856
95,2.496043


In [104]:
forecast_set = pd.concat([forecast_set, display_df],axis=1)

In [87]:
forecast_set

,Date,Price
0,2023-10-21,3.552102
1,2023-10-22,3.540825
2,2023-10-23,3.529284
3,2023-10-24,3.517669
4,2023-10-25,3.506003
...,...,...
92,2024-01-21,2.519781
93,2024-01-22,2.511768
94,2024-01-23,2.503856
95,2024-01-24,2.496043
